Select kernel `segment-anything`

Run this cell to import necessary packages and initialise SAM model and mask generator

In [6]:
# Import numpy for numerical computation, torch for deep learning, matplotlib for plotting, and OpenCV for image processing
import numpy as np
import torch
import matplotlib.pyplot as plt
from segment_anything import sam_model_registry, SamAutomaticMaskGenerator, SamPredictor
import cv2
from ultralytics import YOLO
from PIL import Image
import os
import shutil
from sklearn.model_selection import train_test_split
from IPython.display import display
from IPython.display import clear_output
import yaml
from ultralytics import YOLO

sam_checkpoint = "sam_vit_h_4b8939.pth" # Path to the checkpoint file
model_type = "vit_h" # Model type

device = "cuda" if torch.cuda.is_available() else "cpu" # Use GPU if available, otherwise use CPU

sam = sam_model_registry[model_type](checkpoint=sam_checkpoint) # Load the model
sam.to(device=device) # Move the model to the device

mask_generator = SamAutomaticMaskGenerator(sam) # Create a mask generator

Program Functions

Run to initialise functions

In [5]:
def display_mask_img(mask_img, h, w):
    display(mask_img.resize((w,h)))

def load_images(path):
    images = []
    for filename in os.listdir(path):
        if filename.endswith(".jpg") or filename.endswith(".png"): 
            # complete filename
            img_path = os.path.join(path, filename)
            
            # Read the image
            image = cv2.imread(img_path)
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) # Convert from BGR to RGB

            images.append([image, filename])
    
    return images


def prompt_user(mask_img, h, w):
    #display(mask_img.resize((w,h)))
    legend = ""
    for key, val in labels.items():
        legend += str(key) + ": " + val +'\n'
    while True:
        try:
            user_input = int(input(legend + "Please enter a number: "))
            if 0 <= user_input < len(labels):
                # print(f"You selected: {labels[user_input]}")
                break
            else:
                print("The number is not within the range. Please try again.")
        except ValueError:
            print("That's not a valid number. Please try again.")
    return user_input


# extract the coordinates of the segment from SAM into a list of numpy array
def extract_segment(mask):
    binary_mask = np.array(mask) # get the segmentation of the mask and convert it to a numpy array
    binary_mask = (binary_mask * 255).astype(np.uint8) # convert the mask to a binary mask

    contours, hierarchy = cv2.findContours(binary_mask, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE) # find contours from the binary image
    polygon_coords = [] # stores the coordinates of the vertices of the polygon
    
    for contour in contours:
        epsilon = 0.01 * cv2.arcLength(contour, True) # approximate contour with accuracy proportional to the contour perimeter
        approx = cv2.approxPolyDP(contour, epsilon, True) # approximate contour with the Douglas-Peucker algorithm

        polygon_coords.append(approx) # add the coordinates of the vertices of the polygon to the list
    return polygon_coords


# define a function to filter the mask based on the area
def filter_mask(mask, h, w):
    # specify the minimum area of the mask
    min_area = 0.03 * h * w
    # create a new dictionary
    new_dict = []

    # filter the masks based on the area
    for i in mask:
        if i['area'] > min_area:
            new_dict.append(i)
    
    return new_dict

# display the segment on the image
def draw_segment(img, coord):
    cv2.drawContours(img, coord, -1, (0, 255, 0), 3) # draw the contours on the image
    plt.imshow(img, cmap='gray') # show the image
    plt.show()


def numpy_to_list(numpy_arr):
    # Convert each numpy array in the list to a regular list and extract the inner lists
    coord_list = [coord[0].tolist() for array in numpy_arr for coord in array]
    flat_list = []
    for coord in coord_list:
        flat_list.append(coord[0])
        flat_list.append(coord[1])
    return flat_list


def get_yolo_data(image, masks, h, w):
    masks = filter_mask(masks, h, w)

    data = []
    k = 0

    for i in range(len(masks)):
        image_copy = image.copy()
        bound_box = masks[i]['bbox']
        current_mask = masks[i]['segmentation']
        current_mask_img = image * current_mask[:,:,np.newaxis] + (1 - current_mask[:,:,np.newaxis]) * 255
        current_mask_img = np.uint8(current_mask_img)

        current_mask_img = Image.fromarray(current_mask_img)

        segment_coords = extract_segment(current_mask)
        flat_segment_coords = numpy_to_list(segment_coords)

        for j in range(len(flat_segment_coords)):
            if j%2 == 0:
                flat_segment_coords[j] = flat_segment_coords[j]/w
            else:
                flat_segment_coords[j] = flat_segment_coords[j]/h

        for l in range(4):
            if l%2 == 0:
                bound_box[l] = bound_box[l]/w
            else:
                bound_box[l] = bound_box[l]/h


        
        #display_mask_img(current_mask_img, h, w)
        draw_segment(image_copy, segment_coords)
        label = prompt_user(current_mask_img, h, w)
        clear_output(wait=True)  # this line forces the display of the image before the prompt
        if (labels[label] != "NA"):
            data.append([])
            flat_segment_coords[:0] = bound_box
            flat_segment_coords.insert(0, label)
            data[k].append(flat_segment_coords)
            data[k].append(segment_coords)
            k += 1

    return data


def split_data(src_directory, test_size=0.2):
    os.makedirs('dataset', exist_ok=True)

    os.makedirs(os.path.join('dataset', 'train', 'images'), exist_ok=False)
    os.makedirs(os.path.join('dataset', 'valid', 'images'), exist_ok=False)
    os.makedirs(os.path.join('dataset', 'train', 'labels'), exist_ok=False)
    os.makedirs(os.path.join('dataset', 'valid', 'labels'), exist_ok=False)

    # Get a list of all image file names in the source directory
    all_files = os.listdir(src_directory)

    # Split file names into training and validation sets
    train_files, valid_files = train_test_split(all_files, test_size=test_size)

    # Move files into the train and valid directories
    for file_name in train_files:
        shutil.copy(os.path.join(src_directory, file_name), os.path.join('dataset', 'train', 'images', file_name))
    for file_name in valid_files:
        shutil.copy(os.path.join(src_directory, file_name), os.path.join('dataset', 'valid', 'images', file_name))


def create_yaml(labels, path, train_path="train/images", val_path="valid/images", output_path="dataset/data.yaml"):
    # Create dictionary with desired structure
    data = {
        'names': labels,
        'path': path,
        'train': train_path,
        'val': val_path
    }

    # Write to yaml file
    with open(output_path, 'w') as outfile:
        yaml.dump(data, outfile, default_flow_style=False)

def extract_frames(video_path, output_dir, frame_interval=30):
    os.makedirs(output_dir, exist_ok=True)

    # open video file
    video = cv2.VideoCapture(video_path)

    if not video.isOpened():
        print(f"Could not open video file: {video_path}")
        return
    
    frame_index = 0

    while True:
        # read next frame
        success, frame = video.read()

        # if cannot read frame
        if not success:
            break
            
        # if this frame number is a multiple of frame interval, save it
        if frame_index % frame_interval == 0:
            output_path = os.path.join(output_dir, f"frame_{frame_index}.jpg")
            cv2.imwrite(output_path, frame)
        frame_index += 1

    video.release()

def load_images_from_video(img_path, vid_path):
    # extracts the frames from vid_path and load them to img_path/
    extract_frames(vid_path, img_path)
    split_data(img_path) # splits into train, valid set
                            # outputs to dataset/train/images and dataset/valid/images

    train_images = load_images('dataset/train/images/') # loads 
    valid_images = load_images('dataset/valid/images/')

    train_masks_list = []
    valid_masks_list = []

    for image in train_images:
        h, w = image[0].shape[:2]
        train_masks_list.append([image[0], mask_generator.generate(image[0]),h,w,image[1]]) # Generate masks
        # [ [corresponding image, mask, h, w, filename], ... ]

    for image in valid_images:
        h, w = image[0].shape[:2]
        valid_masks_list.append([image[0], mask_generator.generate(image[0]),h,w,image[1]])

    return train_masks_list, valid_masks_list

def output_to_txt(data, directory):
    fname = data[-1][0:-4]  # get the filename from the last element of the data
    os.makedirs(directory, exist_ok=True)  # creates the directory if it doesn't exist
    with open(os.path.join(directory, fname + '.txt'), 'w') as f:  # joins the directory with the filename
        for line in data[:-1][0]:  # iterate over all elements except for the last one which is the filename
            class_num = line[0][0]  # get the class number
            coords = line[0][1:]  # get the list of coordinates
            f.write(f"{class_num} {' '.join(map(str, coords))}\n")  # write to the file
            
def main_program(train_mask_list, valid_mask_list, labels, dataset_path = r"C:\Users\Adrian\segment-anything\sam-2-yolo\dataset"):
    train_data = []
    valid_data = []

    for mask in train_mask_list:
        train_data.append( [get_yolo_data(mask[0], mask[1], mask[2], mask[3]), mask[4]] )
        # data = [
        #          [
        #            [class, coords, numpy_coords]
        #            ...
        #            [class, coords, numpy_coords], 
        #            filename22
        #          ],  
        #          ...
        #        ]

    for mask in valid_mask_list:
        valid_data.append( [get_yolo_data(mask[0], mask[1], mask[2], mask[3]), mask[4]] )

    for i in range(len(train_data)):
        output_to_txt(train_data[i], 'dataset/train/labels/')
    for j in range(len(valid_data)):
        output_to_txt(valid_data[j], 'dataset/valid/labels/')

    create_yaml(labels, dataset_path)

Input relevant details

Before running this, ensure the workspace has no `dataset` folder

In [20]:
# Specify labels
labels = {
    0: "Pipe",
    1: "Wall",
    2: "Floor",
    3: "Ceiling",
    4: "Door",
    5: "Door frame",
    6: "Gate",
    7: "Building materials",
    8: "Tools",
    9: "Windows",
    10: "Person",
    11: "Safety rails",
    12: "Electrical cable",
    13: "Lens effect",
    14: "NA"
}

# specify where the images will go
img_path = "images/"
# specify where the video to use
vid_path = "stream0_.mkv"

# gets the training masks, and validation masks
train_mask_list, valid_mask_list = load_images_from_video(img_path, vid_path)

This cell will run the main program

In [ ]:
main_program(train_mask_list, valid_mask_list, labels)

# YOLO

In [23]:
# train model with own dataset
model = YOLO('yolov8l-seg.pt')

model.train(data="dataset/data.yaml", epochs=100, imgsz=640)

New https://pypi.org/project/ultralytics/8.0.111 available  Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.0.110  Python-3.10.11 torch-2.0.1 CUDA:0 (NVIDIA GeForce RTX 3080 Ti Laptop GPU, 16384MiB)
yolo\engine\trainer: task=segment, mode=train, model=yolov8l-seg.pt, data=dataset/data.yaml, epochs=100, patience=50, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=None, exist_ok=False, pretrained=False, optimizer=SGD, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=0, resume=False, amp=True, fraction=1.0, profile=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, show=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, vid_stride=1, line_width=None, visualize=False, augment=False, agnostic_nms=F

In [32]:
trained_model = YOLO(r'runs\segment\train2\weights\best.pt')
results = trained_model.predict(source="stream0_.mkv", show=True)



    WARNING  stream/video/webcam/dir predict source will accumulate results in RAM unless `stream=True` is passed,
    causing potential out-of-memory errors for large sources or long-running streams/videos.

    Usage:
        results = model(source=..., stream=True)  # generator of Results objects
        for r in results:
            boxes = r.boxes  # Boxes object for bbox outputs
            masks = r.masks  # Masks object for segment masks outputs
            probs = r.probs  # Class probabilities for classification outputs

video 1/1 (1/3735) C:\Users\Adrian\segment-anything\sam-2-yolo\stream0_.mkv: 608x640 3 Walls, 1 Floor, 1 Ceiling, 1 Windows, 1 Lens effect, 32.0ms
video 1/1 (2/3735) C:\Users\Adrian\segment-anything\sam-2-yolo\stream0_.mkv: 608x640 3 Walls, 1 Floor, 1 Ceiling, 1 Windows, 1 Lens effect, 69.3ms
video 1/1 (3/3735) C:\Users\Adrian\segment-anything\sam-2-yolo\stream0_.mkv: 608x640 3 Walls, 1 Floor, 1 Ceiling, 1 Windows, 1 Lens effect, 40.9ms
video 1/1 (4/3735) C

KeyboardInterrupt: 